In [1]:
!pip install "pymongo[srv]"
!pip install -e .

   ---------------------------------------- 0.0/848.1 kB ? eta -:--:--
   --------------------------------------- 848.1/848.1 kB 18.9 MB/s eta 0:00:00

   ---------------------------------------- 0/2 [dnspython]
   ---------------------------------------- 0/2 [dnspython]
   ---------------------------------------- 0/2 [dnspython]
   ---------------------------------------- 0/2 [dnspython]
   ---------------------------------------- 0/2 [dnspython]
   ---------------------------------------- 0/2 [dnspython]
   -------------------- ------------------- 1/2 [pymongo]
   -------------------- ------------------- 1/2 [pymongo]
   -------------------- ------------------- 1/2 [pymongo]
   -------------------- ------------------- 1/2 [pymongo]
   -------------------- ------------------- 1/2 [pymongo]
   -------------------- ------------------- 1/2 [pymongo]
   -------------------- ------------------- 1/2 [pymongo]
   -------------------- ------------------- 1/2 [pymongo]
   ---------------------

Obtaining file:///C:/Users/rober/OneDrive/Desktop/Programmieren/torch_vscode/rag


ERROR: file:///C:/Users/rober/OneDrive/Desktop/Programmieren/torch_vscode/rag does not appear to be a Python project: neither 'setup.py' nor 'pyproject.toml' found.


In [3]:
!pip install -qU langchain-mongodb pymongo
!pip install -q langchain
!pip install -U langchain-community
!pip install -q transformers
!pip install -q sentence-transformers
!pip install -q faiss-cpu

In [ ]:

# mit online Datenbank MongoDB Atlas verbinden

#!! damit es funktioniert:
# - diese ip adresse in atlas hinterlegen
# - evtl neuen user mit admin rechte erstellen wenn fehler 8000

from pymongo.mongo_client import MongoClient
from pymongo.server_api import ServerApi


import os
from dotenv import load_dotenv
load_dotenv()

uri = os.getenv("MONGODB_ATLAS_CLUSTER_URI")

client = MongoClient(uri, server_api=ServerApi('1'))
try:
    client.admin.command('ping')
    print("Pinged your deployment. You successfully connected to MongoDB!")
except Exception as e:
    print(e)



In [52]:
from langchain.embeddings import HuggingFaceEmbeddings
# Define the path to the pre-trained model you want to use
modelPath = "sentence-transformers/all-MiniLM-l6-v2"
modelPath = "BAAI/bge-large-en-v1.5"

# Create a dictionary with model configuration options, specifying to use the CPU for computations
model_kwargs = {'device':'cuda'}

# Create a dictionary with encoding options, specifically setting 'normalize_embeddings' to False
encode_kwargs = {'normalize_embeddings': True}

# Initialize an instance of HuggingFaceEmbeddings with the specified parameters
embeddings = HuggingFaceEmbeddings(
    model_name=modelPath,     # Provide the pre-trained model's path
    model_kwargs=model_kwargs, # Pass the model configuration options
    encode_kwargs=encode_kwargs # Pass the encoding options
)


emb = embeddings.embed_query("Hello world")
embeddings_dim = len(emb)
print("embedding dim: ", embeddings_dim, sum(emb))

c:\Users\rober\OneDrive\Desktop\Programmieren\torch_vscode\torch_venv\Lib\site-packages\huggingface_hub\file_download.py:142: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\rober\.cache\huggingface\hub\models--BAAI--bge-large-en-v1.5. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


embedding dim:  1024 -0.19330515984938756


In [32]:
from langchain_mongodb import MongoDBAtlasVectorSearch
from pymongo import MongoClient

# index already created, no need to create it again for this collection

# initialize MongoDB python client
client = MongoClient(uri)

DB_NAME = "rag"
COLLECTION_NAME = "raggyshaggy"
ATLAS_VECTOR_SEARCH_INDEX_NAME = "index-384"

MONGODB_COLLECTION = client[DB_NAME][COLLECTION_NAME]
print(MONGODB_COLLECTION)

vector_store = MongoDBAtlasVectorSearch(
    collection=MONGODB_COLLECTION,
    embedding=embeddings,
    index_name=ATLAS_VECTOR_SEARCH_INDEX_NAME,
    relevance_score_fn="cosine",
)

# Create vector search index on the collection
# Since we are using the default OpenAI embedding model (ada-v2) we need to specify the dimensions as 1536
vector_store.create_vector_search_index(dimensions=embeddings_dim)

Collection(Database(MongoClient(host=['ac-wsbimpq-shard-00-01.ahwsyk2.mongodb.net:27017', 'ac-wsbimpq-shard-00-00.ahwsyk2.mongodb.net:27017', 'ac-wsbimpq-shard-00-02.ahwsyk2.mongodb.net:27017'], document_class=dict, tz_aware=False, connect=True, retrywrites=True, w='majority', appname='ragcluster', authsource='admin', replicaset='atlas-t1ht9f-shard-0', tls=True), 'rag'), 'raggyshaggy')


In [33]:
import pandas as pd
from langchain_core.documents import Document
# Set the path to the file you'd like to load
file_path = "CNN_Articels_small.csv"

df = pd.read_csv(file_path)
print("First 5 records:", df.head())

# Function to convert dataframe row to Langchain Document
def row_to_document(row):
    # Customize this based on your dataframe structure
    # 'page_content' will typically be the main text column
    # 'metadata' can include other relevant columns
    page_content = row['Article text'] # Replace 'Article' with the name of your text column
    if pd.isna(page_content):
      print("!!!!kein artikeltext")
      page_content = row['Headline']
    metadata = {
        'Index': row['Index'],
        'Author': row['Author'],
        'Date published': row['Date published'],
        'Category': row['Category'],
        'Section': row['Section'],
        'Url': row['Url'],
        'Headline': row['Headline'],
        'Description': row['Description'],
        'Keywords': row['Keywords'],
        'Second headline': row['Second headline']

        # Add more metadata fields as needed
    }
    return Document(page_content=page_content, metadata=metadata)

# Convert the dataframe to a list of Langchain Document objects
df_documents = df.apply(row_to_document, axis=1).tolist()

# Print the first few converted documents to verify
print(df_documents[:5])

First 5 records:    Index                                             Author  \
0      0                                 Jacopo Prisco, CNN   
1      2                              Stephanie Bailey, CNN   
2      3  Words by Stephanie Bailey, video by Zahra Jamshed   
3      4                    Paul R. La Monica, CNN Business   
4      7                                            Reuters   

        Date published  Category    Section  \
0  2021-07-15 02:46:59      news      world   
1  2021-05-12 07:52:09      news      world   
2  2021-06-16 02:51:30      news       asia   
3  2022-03-15 09:57:36  business  investing   
4  2022-03-15 11:27:02  business   business   

                                                 Url  \
0  https://www.cnn.com/2021/07/14/world/tusimple-...   
1  https://www.cnn.com/2021/05/12/world/ironhand-...   
2  https://www.cnn.com/2021/06/15/asia/swarm-robo...   
3  https://www.cnn.com/2022/03/15/investing/brics...   
4  https://www.cnn.com/2022/03/15/busines

In [27]:
from langchain_community.retrievers import BM25Retriever
from langchain.text_splitter import RecursiveCharacterTextSplitter

#data = loader.load()
data = df_documents

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=150)
# 'data' holds the text you want to split, split the text into documents using the text splitter.
docs = text_splitter.split_documents(data)
#retriever = BM25Retriever.from_documents(docs)


In [ ]:
from uuid import uuid4

uuids = [str(uuid4()) for _ in range(len(docs))]

vector_store.add_documents(documents=docs, ids=uuids)

['42f85094-7098-437f-ac99-4935ac7eba04',
 'd3c49487-c93c-4c56-a3a5-a56b40ac168a',
 'd7b8f7e3-6958-4f34-be72-95001846a5b0',
 '4767fb51-9993-40c1-88e9-8659e8a248e3',
 '7395d637-55d4-4bbe-a6ea-39b95087e449',
 'bdd87d15-a36d-466a-9fb0-26d0b444df24',
 '34500fe4-6602-436d-9f7e-15b1a36a657a',
 '942edaf8-ae89-4090-a0f3-0cae33239578',
 '68203dcf-e5c9-4403-84a3-cbc341ca6af4',
 'b1a4ef14-d76b-4294-9fc5-1dfd8eb79f7e',
 'dbf9dc5c-2b9a-44ac-b99c-a5bc093b05a1',
 'ae523595-409c-4e11-9b57-f3b1b288e28c',
 '68cfeb36-ca35-4d80-ae0d-30eb8bdb4dfb',
 '0503c3b0-3315-448d-9e94-5d4216562d1a',
 '800a4a09-a68c-4d6d-ace5-9331e4481978',
 'fc6922b6-0205-4b58-ab7f-63c1d56c7cfe',
 '57f7e3ba-211e-4cdc-8a5e-48a74e54047c',
 '20973232-a35a-48db-9ad1-2ad11d815b0a',
 '2a88878b-24b6-4c7b-8792-519b0f042c7e',
 '367a3259-8feb-49f6-b5cc-24313e1a3c36',
 '96810ca6-7902-47d1-9270-71f677213b9f',
 'da5f4af5-528e-4aa0-8791-e57fc90b8c82',
 '77318d6d-ab22-489b-a534-255896e2a36d',
 'ff4f1624-5bb2-4d3d-83e6-699e1bc7d13a',
 '7ec53153-f233-

In [47]:
results = vector_store.similarity_search(
    "How many gold medals did Simone Biles win in the summer olympics 2016?", k=5
)
for res in results:
    print(f"* {res.page_content} [{res.metadata}]")

* 3. He would go on to win gold.Hide Caption 85 of 312 Photos: The best photos from the Tokyo OlympicsJamaica's Elaine Thompson-Herah reacts after defending her crown in the 200 meters on August 3. She also won gold in the 100 meters on Saturday. She's the first-ever woman to win the 100 and 200 double at consecutive Olympic Games.Hide Caption 86 of 312 Photos: The best photos from the Tokyo OlympicsUS gymnast Simone Biles is congratulated by coach Cecile Canqueteau-Landi as it became evident that Biles would earn a medal on August 3. Biles now has seven Olympic medals, tying her with Shannon Miller for the most by an American gymnast.Hide Caption 87 of 312 Photos: The best photos from the Tokyo OlympicsGerman sailors Erik Heil and Thomas Ploessel jump into the water as they celebrate winning bronze in the 49er category on August 3.Hide Caption 88 of 312 Photos: The best photos from the Tokyo OlympicsAthletes compete in a 400-meter heat on August 3.Hide Caption 89 of 312 Photos: The [{

In [36]:
results = vector_store.similarity_search_with_score("How many gold medals did Simone Biles win in the summer olympics 2016?", k=5)
for res, score in results:
    print(f"* [SIM={score:3f}] {res.page_content} [{res.metadata}]")

* [SIM=0.870440] 3. He would go on to win gold.Hide Caption 85 of 312 Photos: The best photos from the Tokyo OlympicsJamaica's Elaine Thompson-Herah reacts after defending her crown in the 200 meters on August 3. She also won gold in the 100 meters on Saturday. She's the first-ever woman to win the 100 and 200 double at consecutive Olympic Games.Hide Caption 86 of 312 Photos: The best photos from the Tokyo OlympicsUS gymnast Simone Biles is congratulated by coach Cecile Canqueteau-Landi as it became evident that Biles would earn a medal on August 3. Biles now has seven Olympic medals, tying her with Shannon Miller for the most by an American gymnast.Hide Caption 87 of 312 Photos: The best photos from the Tokyo OlympicsGerman sailors Erik Heil and Thomas Ploessel jump into the water as they celebrate winning bronze in the 49er category on August 3.Hide Caption 88 of 312 Photos: The best photos from the Tokyo OlympicsAthletes compete in a 400-meter heat on August 3.Hide Caption 89 of 312

In [45]:
retriever = vector_store.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={"k": 5, "score_threshold": 0.2},
)
retriever.invoke("eerm")

[Document(id='b0007711-b83f-402d-a60c-8e82e0dda272', metadata={'_id': 'b0007711-b83f-402d-a60c-8e82e0dda272', 'Index': 4195, 'Author': 'Rob Picheta and Matias Grez, CNN', 'Date published': '2021-11-21 10:48:11', 'Category': 'sport', 'Section': 'football', 'Url': 'https://www.cnn.com/2021/11/21/football/ole-gunnar-solskjaer-manchester-united-departure-spt-intl/index.html', 'Headline': 'Ole Gunnar Solskjaer sacked as Manchester United manager after string of poor results - CNN', 'Description': 'Manchester United have confirmed the departure of manager Ole Gunnar Solskjaer, one day after the club suffered a humiliating 4-1 Premier League defeat at Watford.', 'Keywords': 'football, Ole Gunnar Solskjaer sacked as Manchester United manager after string of poor results - CNN', 'Second headline': 'Ole Gunnar Solskjaer out as Manchester United manager after string of poor results'}, page_content='pressure had been mounting on the Norwegian coach for several weeks, but the team was able to pull 

In [43]:
# actually call the retriever

query = "How many gold medals did Simone Biles win in the summer olympics 2016?"
#query = "What is the impact of AI on journalism?"


results = retriever.invoke(query)
# Print the results
print("Retrieved Documents:")
for doc in results:
    print(f"Text: {doc.page_content}\nMetadata:\n")
    for k, v in doc.metadata.items():
        print(f"  {k}: {v}")
    print()


Retrieved Documents:
Text: 3. He would go on to win gold.Hide Caption 85 of 312 Photos: The best photos from the Tokyo OlympicsJamaica's Elaine Thompson-Herah reacts after defending her crown in the 200 meters on August 3. She also won gold in the 100 meters on Saturday. She's the first-ever woman to win the 100 and 200 double at consecutive Olympic Games.Hide Caption 86 of 312 Photos: The best photos from the Tokyo OlympicsUS gymnast Simone Biles is congratulated by coach Cecile Canqueteau-Landi as it became evident that Biles would earn a medal on August 3. Biles now has seven Olympic medals, tying her with Shannon Miller for the most by an American gymnast.Hide Caption 87 of 312 Photos: The best photos from the Tokyo OlympicsGerman sailors Erik Heil and Thomas Ploessel jump into the water as they celebrate winning bronze in the 49er category on August 3.Hide Caption 88 of 312 Photos: The best photos from the Tokyo OlympicsAthletes compete in a 400-meter heat on August 3.Hide Caption